In [204]:
import cvxpy as cp
import numpy as np

# First, we write the function for creating box
# This is the code to make a box around a vector v = [v1 v2 ... vn]^T with parameters r = [r1 r2 ... rn]^T
def make_box(v, r):
    n = len(v)
    zeros = [0]*(n+1)
    mat = []
    # Adding the constraints for positive values
    for r1 in r:
        zeros[0] = r1
        mat.append(zeros*1)
    # Adding the constraints for negative values
    for r1 in r:
        zeros[0] = r1
        mat.append(zeros*1)   
    # print('mat zeros = ', mat) # [[0.1,0,0], [0.1,0,0], [0.1,0,0], [0.1,0,0]]
    
    for i in range(n):
        mat[i][i+1] = 1 # [[0.1,1,0], [0.1,0,1], [0.1,0,0], [0.1,0,0]]
    # print('mat pos = ', mat)
    j = 1
    for i in range(n,2*n):
        mat[i][j] = -1 # # [[0.1,1,0], [0.1,0,1], [0.1,-1,0], [0.1,0,-1]]
        j += 1
    # print('mat for constraints = ', mat)
    Box = Polyhedron(ieqs=mat, backend='ppl', base_ring=QQ)
    Box_vertices = np.array(Box.vertices_list())
    return Box_vertices

# We write a function to compute minimum distance between two polyhedra, and return the points
def min_dist_between_polyhedra(P1, P2):
    # At first, we extract the linear constraints from both polyhedra written as Ax + b >= 0
    Ab1 = list(P1.Hrepresentation())
    Ab2 = list(P2.Hrepresentation())
    b1, A1 = ([a[0] for a in Ab1], [a[1:] for a in Ab1])
    b2, A2 = ([a[0] for a in Ab2], [a[1:] for a in Ab2])
    b1, A1, b2, A2 = np.array(b1), np.array(A1), np.array(b2), np.array(A2)
    # print('b1 = ', b1)
    # print('A1 = ', A1)
    # print('b2 = ', b2)
    # print('A2 = ', A2)

    # The dimension of the polyhedra are the number of columns in Matrix A
    n = len(A1[0])
    print('n = ', n)

    # We define two vectors of same dimensions
    X1 = cp.Variable(n)
    X2 = cp.Variable(n)

    # Next, we write the constraints for the two polyhedra P1 and P2 such that vector X1 in P1 and X2 in P2
    constraints = [A1@X1 >= -b1, A2@X2 >=-b2]

    # Finally, we write the objective function which is to minimize the infinity norm of any two points in P1 and P2
    objective = cp.Minimize(cp.norm_inf(X1-X2))

    # Problem formulation
    prob = cp.Problem(objective, constraints)

    # Solve the problem
    prob.solve()

    # Finally, we have the minimum distance and the two vectors,
    val, x1, x2 = prob.value, X1.value, X2.value
    # print('Minimum distance = ', val)
    # print('Vector in P1 = ', x1)
    # print('Vector in P2', x2)
    return val, x1, x2

# Next, we write a function to compute the maximum distance of a vector and a polyhedron
def max_dist_vector_to_polyhedron(v, P):
    X = P.vertices_list()
    dist_list = [(np.linalg.norm(v-x), x) for x in X]
    d, vx = max(dist_list)
    return d, vx

# Minkowski sum function
def minkowski_pol(V1, V2):
    V3 = []
    for v1 in V1:
        for v2 in V2:
            n = v1.size
            v = np.zeros(n)
            for i in range(n):
                v[i] = v1[i] + v2[i]
            V3.append(v)
    # print(V3)
    V4 = np.array(V3)
    return V4

def convex_hull(V1, V2):
    V3 = V1 + V2
    return V3

def linear_trans(M, V):
    # V1 = P1.vertices_list()
    # V2 = P2.vertices_list()
    V1 = []
    for v in V:
        x =  np.dot(M,v)
        x = np.array(x)
        V1.append(x)
        # print(V1)
    V2 = np.array(V1)
    return V2

In [206]:
# This is the input for anytime perception of a made-up example

# We will describe a simple 2D example. First we define the initial state and the initial input
# X = [x y] is a state in 2D
# U = [u] is an input in 1D
PX0 = Polyhedron(ieqs=[(-1,0,1),(4,0,-1),(-2,3,-1),(2,-1,1)], backend='ppl', base_ring=QQ)
PU0 = Polyhedron(eqns=[(-3,1)], backend='ppl', base_ring=QQ)

X0 = np.array(PX0.vertices_list())
U0 = np.array(PU0.vertices_list())

P0 = [np.hstack((x,u,1)) for x in X0 for u in U0]

# Since we have X in 2D and U in 1D, A will be a 2x2 matrix, B will be a 2x1 matrix, and K will be a 1x2 matrix
A = np.array([[1,2],[3,4]])
B = np.array([[0],[1]])
# K = np.array([8, 11])
K = np.array([[-3,-5]])

# The dimension and epsilon for the box
xn, ep = 2, 0.2
xu = 1

# Parameters for alg3
T0 = [2,9]
l, u = T0
# W = Polyhedron(ieqs=[(-10,0,1),(40,0,-1),(-20,3,-1),(20,-1,1)], backend='ppl', base_ring=QQ) # Unsafe state
c = 5
C = A*0.5
D = B*2
E = np.array([[2],[1]])
print("X0 = ", X0)
print("U0 = ", U0)
print("P0 = ", P0)
W = [[10,10],[10,20],[20,20],[20,10]]
W = Polyhedron(vertices=W)
I = np.identity(n)
M = Matrix(np.vstack((I,I)))


X0 =  [[3 1]
 [6 4]
 [2 4]
 [1 1]]
U0 =  [[3]]
P0 =  [array([3, 1, 3, 1]), array([6, 4, 3, 1]), array([2, 4, 3, 1]), array([1, 1, 3, 1])]


In [211]:
# Converting K into K = [[I 0] [0 K]]

K1 = [[0 for i in range(xn)]for j in range(xn)]
KI = np.identity(xn)
K1 = np.hstack((KI, K1))
K2 = [[0 for i in range(xn)] for j in range(xu)]
K2 = np.hstack((K2,K))
Kp = np.array(np.vstack((K1,K2)))
print('Kp = ', Kp)

Kp =  [[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -3. -5.]]


In [212]:
# AlC = A*l + C
# BlD = B*l + D
# AuC = A*u + C
# BuD = B*u + D
# J1 = np.hstack((AlC,BlD,E))
# J2 = np.hstack((AuC,BuD,E))
# print('J1 = ', J1)
# print('J2 = ', J2)
# print('P0 = ', P0)
# X1 = [np.dot(J1,p) for p in P0]
# print('X1 = ', X1)
# X2 = [np.dot(J2,p) for p in P0]
# print('X2 = ', X2)
# X = X1 + X2
# print('X = ', X)
# ep = c/l
# print('ep = ', ep)
# X_twice = [np.dot(M,x) for x in X]
# print('X_twice = ', X_twice)
# rbox = [0]*n +[ep]*n
# Box = make_box(vbox, rbox)
# Xh = minkowski_pol(X_twice, Box)
# print("Xh = ", Xh)
# P1 = [np.dot(Kp, x) for x in Xh]
# print('P1 = ', P1)
# P = [np.hstack((p,1)) for p in P1]
# print('P = ', P)
# Xh_poly = Polyhedron(vertices = Xh)
# print(Xh_poly.vertices_list())
# l, x1, w1 = min_dist_between_polyhedra(Xh_poly, W)
# u, x2 = max_dist_vector_to_polyhedron(w1, Xh_poly)
# print('l, x1, w1, u, x2')
# print(l,x1,w1,u,x2)

In [213]:
AlC = A*l + C
BlD = B*l + D
AuC = A*u + C
BuD = B*u + D
J1 = np.hstack((AlC,BlD,E))
J2 = np.hstack((AuC,BuD,E))
print('J1 = ', J1)
print('J2 = ', J2)
print('P0 = ', P0)
X1 = [np.dot(J1,p) for p in P0]
print('X1 = ', X1)
X2 = [np.dot(J2,p) for p in P0]
print('X2 = ', X2)
X = X1 + X2
print('X = ', X)
ep = c/l
print('ep = ', ep)
X_twice = [np.dot(M,x) for x in X]
print('X_twice = ', X_twice)
vbox = [0]*(2*n)
rbox = [0]*n +[ep]*n
Box_b = make_box(vbox, rbox)
Xh = minkowski_pol(X_twice, Box_b)
print("Xh = ", Xh)
P1 = [np.dot(Kp, x) for x in Xh]
print('P1 = ', P1)
P = [np.hstack((p,1)) for p in P1]
print('P = ', P)
Xh_poly = Polyhedron(vertices = Xh)
print(Xh_poly.vertices_list())
vbox, rbox = [0]*n, [ep]*n
Box_a = make_box(vbox,rbox)
Xpp = X+Box_a
l, x1, w1 = min_dist_between_polyhedra(Xpp, W)
u, x2 = max_dist_vector_to_polyhedron(w1, Xpp)
print('l, x1, w1, u, x2')
print(l,x1,w1,u,x2)

J1 =  [[ 2.5  5.   0.   2. ]
 [ 7.5 10.   4.   1. ]]
J2 =  [[ 9.5 19.   0.   2. ]
 [28.5 38.  11.   1. ]]
P0 =  [array([3, 1, 3, 1]), array([6, 4, 3, 1]), array([2, 4, 3, 1]), array([1, 1, 3, 1])]
X1 =  [array([14.5, 45.5]), array([37., 98.]), array([27., 68.]), array([ 9.5, 30.5])]
X2 =  [array([ 49.5, 157.5]), array([135., 357.]), array([ 97., 243.]), array([ 30.5, 100.5])]
X =  [array([14.5, 45.5]), array([37., 98.]), array([27., 68.]), array([ 9.5, 30.5]), array([ 49.5, 157.5]), array([135., 357.]), array([ 97., 243.]), array([ 30.5, 100.5])]
ep =  5/2
X_twice =  [array([14.5, 45.5, 14.5, 45.5]), array([37., 98., 37., 98.]), array([27., 68., 27., 68.]), array([ 9.5, 30.5,  9.5, 30.5]), array([ 49.5, 157.5,  49.5, 157.5]), array([135., 357., 135., 357.]), array([ 97., 243.,  97., 243.]), array([ 30.5, 100.5,  30.5, 100.5])]
Xh =  [[ 14.5  45.5  12.   48. ]
 [ 14.5  45.5  12.   43. ]
 [ 14.5  45.5  17.   43. ]
 [ 14.5  45.5  17.   48. ]
 [ 37.   98.   34.5 100.5]
 [ 37.   98.   34.5 

ValueError: operands could not be broadcast together with shapes (8,2) (4,2) 